# Transparent mortgage calculator

Are you considering getting a loan? Do you already have a loan? Do you want to know more about the logic behind your mortgage's calculation and have more metrics that tell you how much money (in totals) are you paying for your loan? 

Then this calculator may be for you.

The transparent mortgage calculator script provides allows the user to model a mortgage and understand the credit costs associated with it.

In the following section, I introduce some basic concepts necessary to understand mortgage calculations and how banks handle them.

Please bear in mind that the script does not account for administrative fees that the user might have to pay to the bank for handling the mortgage or for the purchase of the property (land tax, register's or notary fees).

## Mortgage key terms:

*   Interest rate:

*   Principal:


## Mortgage logic:

Banks may use 2 types of logic for handling mortgages:

*  **Fixed principal style**: you decide to pay exactly 1,000 EUR of your debt every month, plus whatever interest is owed. As a result, your debt goes down by exactly 1,000 every month, your interest goes down too. In this model you pay off the principal faster because you are paying a flat amount of debt plus interest

*  Fixed Total Payment Style (The Mortgage Standard), also known as **French Amortization System**: You pay exactly 1,000 EUR every single month for 30 years. At the beginning, that 1,000 is mostly interest. As the years go by, the interest portion shrinks, and the principal portion grows to fill the gap.

This code uses the French Amortization System for handling the logic of the mortgage calculation

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


In [ ]:
#Function

def mort ():

  #Obtaining total cost of property from end user
  while True:
    try:
      total = float(input("Introduce total cost of the property: "))
      print()
      break

    except ValueError:
      print("Error, enter only numbers")
      print()

  #Obtaining down payment from end user
  while True:
    try:
      down_py = float(input("Introduce down payment: "))
      if down_py > total:
        print(f"Your loan amount cannot be bigger than the total cost of the property")
        print()
        continue
      break

    except ValueError:
      print("Error, enter only numbers")
      print()

  loan_amount = total - down_py
  print(f"Your loan amount is {loan_amount}\n")

  #Obtaining interests - Ideally API call

  while True:
    try:
      print(f"Today's 6 months EURIBOR average is XXX")
      Euribor = float(input("Introduce your EURIBOR rate, if none, use 0: "))
      break
    except ValueError:
      print("Error, enter only numbers")

  while True:
    try:
      bank_int = float(input("Introduce your banks interest rate (without Euribor): "))
      break
    except ValueError:
      print("Error, enter only numbers")

  total_interest = Euribor + bank_int
  print(f"Your compound interest is {total_interest}\n")

  #Obtaining first payment date
  while True:
    try:

      first_payment_date_input = input("Introduce the first payment date in this format DD/MM/YYYY: ")
      first_payment_date = datetime.strptime(first_payment_date_input, "%d/%m/%Y")
      break

    except ValueError:
     print("Error: use format DD/MM/YYYY")

  #Obtaining lenght of the mortgage and creating a timeline

  while True:
    try:
      years_payment = int(input("Introduce total number of years you want to pay for: "))
      print()
      break
    except ValueError:
     print("Error: it has to be a round number of years")
     print()

  last_payment_date = first_payment_date + relativedelta(years= years_payment)  #calculating the end date of the payment

  r = relativedelta(last_payment_date, first_payment_date)                      #calculating all the months
  total_months_payments = r.years * 12 + r.months

  print(f"Your first payment date would be on {first_payment_date.strftime('%d-%m-%Y')} and the last one on {last_payment_date.strftime('%d-%m-%Y')}, a total of {total_months_payments} months\n")

  # Creating a table with all the calculations

  rows = []

  current_date = first_payment_date                                             # to be used for iteration, avoid changing original
  current_balance = loan_amount
  cumulative_interest = 0
  cumulative_principal = 0

  n = total_months_payments                                                     # to be used for iteration, avoid changing original
  r = (total_interest / 100) / 12                                               #monthly interest rate

  paid_months = 1                                                               # to be used for iteration

  monthly_payment = loan_amount * (r * (1 + r)**n) / ((1 + r)**n - 1)           #Standard Amortization Formula


  while current_date <= last_payment_date:

    paid_total = monthly_payment * paid_months
    interest_this_month = current_balance * r
    principal_this_month =  monthly_payment - interest_this_month

    cumulative_interest += interest_this_month
    cumulative_principal += principal_this_month

    current_balance -= principal_this_month


    new_row = {

        'payment_date': current_date.strftime('%d/%m/%Y'),
        'initial_total_mortgage': loan_amount,
        'monthly_payment': monthly_payment,
        'total_paid': paid_total,
        'current_balance': current_balance,
        'interest_payment_month': interest_this_month,
        'principal_payment_month': principal_this_month,
        'total_paid_in_interest': cumulative_interest,
        'total_paid_in_principal': cumulative_principal

      }

    rows.append(new_row)

    current_date += relativedelta(months=1)
    total_months_payments -= 1
    paid_months +=1

  df = pd.DataFrame(rows)

  total_paid_interest = df['total_paid_in_interest'][-1]
  total_paid_loan = df['total_paid_in_principal'][-1]
  total_paid = total_paid_interest + total_paid_loan

  print(f"Your monthly payment would be of {monthly_payment}/n")
  print(f"By the end of it, you'd pay {total_paid_interest} in interest, {total_paid_loan} in principals for the loan, and {total_paid} in total\n")
  #print(df.head(10))

  #Visualising the results, we want to present them on a yearly basis so we need to transform the data a bit
  df['payment_date'] = pd.to_datetime(df['payment_date'], dayfirst=True)

  df_yearly = df.groupby(df['payment_date'].dt.year).agg({
    'total_paid_in_interest': 'last',
    'total_paid_in_principal': 'last',
    'current_balance': 'last'
}).reset_index()

  import matplotlib.pyplot as plt

  plt.figure(figsize= (14, 6))
  plt.plot(df.payment_date, df.total_paid_in_interest, label = 'Total Paid in Interest')
  plt.plot(df.payment_date, df.total_paid_in_principal, label = 'Total Paid in Principal')
  plt.xlabel('Years')
  plt.ylabel('Total Paid')
  plt.show()




In [ ]:
# Let's call the function

mort()